In [1]:
# Full ML pipeline for AZUREML

from azureml.core import Workspace, Experiment, Run,Datastore,Dataset, Run, Environment
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.steps import PythonScriptStep
from azureml.core import RunConfiguration

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (pywin32 302 (c:\users\satya\anaconda3\envs\tf-gpu-cuda8\lib\site-packages), Requirement.parse('pywin32==227; sys_platform == "win32"'), {'docker'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (pywin32 302 (c:\users\satya\anaconda3\envs\tf-gpu-cuda8\lib\site-packages), Requirement.parse('pywin32==227; sys_platform == "win32"'), {'docker'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (pywin32 302 (c:\users\satya\anaconda3\envs\tf-gpu-cuda8\lib\site-packages), Requirement.parse('pywin32==227; sys_platform == "win32"'), {'docker'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml

In [2]:
#ws=Workspace.create(name='defaultscredits', subscription_id='7b1b43ca-4b64-43cf-9446-edb35a04d7d1', resource_group='defaultsrg01', location='eastus')
ws=Workspace.from_config(path='C:/Users/satya/Downloads/Azure/config.json')

In [7]:
external_datastore_access=Datastore.register_azure_blob_container(ws,'creditdatastore',container_name='credit',account_key='xNVGcQcEXvuuJoWfBpTVpaCAg5ccuwSU4+OObAa0q+M30pif/eZXTrIXQLY9g+eWqDTLxZ8XoRdB+AStzYaxtQ==',account_name='creditstorage12')

In [3]:
external_datastore=Datastore.get(ws, 'creditdatastore')
csv=[(external_datastore,'UCI_Credit_Card.csv')]
credit_dataset=Dataset.Tabular.from_delimited_files(csv)
credit_dataset=credit_dataset.register(ws,credit_dataset,'credit_dataset')

In [5]:
#checks to ensure azure dataset is correctly loaded
credit_dataset.to_pandas_dataframe().head(1)

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1


In [4]:
from azureml.pipeline.steps import PythonScriptStep


In [5]:
from azureml.core.runconfig import RunConfiguration
#import CondaDependencies
from azureml.core.conda_dependencies import CondaDependencies


myenv=Environment(name='myenv')
dep=CondaDependencies.create(conda_packages=['numpy','pandas','scikit-learn'])
myenv.python.conda_dependencies=dep
myenv.register(ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20220616.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "myenv",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
        

In [1]:
#impport AMLCompute
from azureml.core.compute import ComputeTarget, AmlCompute
#import PythonScriptStep
from azureml.pipeline.steps import PythonScriptStep
cluster_name='amlcompute1'
cluster_config=AmlCompute.provisioning_configuration(vm_size='Standard_D4as_v4',max_nodes=1)
cluster=AmlCompute.create(ws,cluster_name,cluster_config)

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (pywin32 302 (c:\users\satya\anaconda3\envs\tf-gpu-cuda8\lib\site-packages), Requirement.parse('pywin32==227; sys_platform == "win32"'), {'docker'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (pywin32 302 (c:\users\satya\anaconda3\envs\tf-gpu-cuda8\lib\site-packages), Requirement.parse('pywin32==227; sys_platform == "win32"'), {'docker'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (pywin32 302 (c:\users\satya\anaconda3\envs\tf-gpu-cuda8\lib\site-packages), Requirement.parse('pywin32==227; sys_platform == "win32"'), {'docker'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml

ComputeTargetException: ComputeTargetException:
	Message: Invalid provisioning configuration, max nodes count is mandatory
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Invalid provisioning configuration, max nodes count is mandatory"
    }
}

In [7]:
#runconfigs
run_config=RunConfiguration()
run_config.target=cluster
run_config.environment=myenv

In [17]:
#creating a reference datafolder
datafolder=PipelineData('datafolder',datastore=external_datastore)

data_step=PythonScriptStep(name="preparation", source_directory='.',script_name='prep1.py',runconfig=run_config,inputs=[credit_dataset.as_named_input('raw_data')],outputs=[datafolder],arguments=['--datafolder',datafolder])
train_step=PythonScriptStep(name='training',source_directory='.',script_name='train1.py',runconfig=run_config,inputs=[datafolder],arguments=['--datafolder',datafolder])



In [18]:
steps=[data_step,train_step]
experiment=Experiment(ws,'credit_pipeline_experiment')
pipeline_credit=Pipeline(ws,steps=steps)
run_submit=experiment.submit(pipeline_credit)
run_submit.wait_for_completion(show_output=True)

Created step preparation [d17b009f][721ac420-c44d-4c40-9bb8-cdf9e45e2545], (This step will run and generate new outputs)
Created step training [0412b06c][f1586355-7f09-43ef-9902-40d960f6a4fe], (This step will run and generate new outputs)
Submitted PipelineRun d1be6e69-b8bb-42b5-ad1d-7010e55539bd
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/d1be6e69-b8bb-42b5-ad1d-7010e55539bd?wsid=/subscriptions/7b1b43ca-4b64-43cf-9446-edb35a04d7d1/resourcegroups/defaultsrg01/workspaces/defaultscredits&tid=a287f42c-46eb-424f-ab40-cd784a7b423c
PipelineRunId: d1be6e69-b8bb-42b5-ad1d-7010e55539bd
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/d1be6e69-b8bb-42b5-ad1d-7010e55539bd?wsid=/subscriptions/7b1b43ca-4b64-43cf-9446-edb35a04d7d1/resourcegroups/defaultsrg01/workspaces/defaultscredits&tid=a287f42c-46eb-424f-ab40-cd784a7b423c
PipelineRun Status: Running


Expected a StepRun object but received <class 'azureml.core.run.Run'> instead.
This usually indicates a package conflict with one of the dependencies of azureml-core or azureml-pipeline-core.
Please check for package conflicts in your python environment


Expected a StepRun object but received <class 'azureml.core.run.Run'> instead.
This usually indicates a package conflict with one of the dependencies of azureml-core or azureml-pipeline-core.
Please check for package conflicts in your python environment


: 

In [8]:
# import automlconfig
from azureml.train.automl import AutoMLConfig

In [9]:
automl_config=AutoMLConfig(task='classification', 
compute_target=cluster,
training_data=credit_dataset,
validation_size=0.3,
label_column_name='default.payment.next.month',
primary_metric='accuracy',
iterations=10,
max_concurrent_iterations=2, experiment_timeout_hours=0.25, featurization='auto')



In [10]:
experiment=Experiment(ws,'credit_automl_experiment')
run=experiment.submit(automl_config)
run.wait_for_completion(show_output=True)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
credit_automl_experiment,AutoML_27a228e2-7485-4cd2-b961-70749e5dba68,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


Experiment,Id,Type,Status,Details Page,Docs Page
credit_automl_experiment,AutoML_27a228e2-7485-4cd2-b961-70749e5dba68,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       DONE
DESCRIPTION:  If the missing values are expected, let the run complete. Otherwise cancel the current run and use a script to customize the handling of missing feature values that may be more appropriate based on the data type and business requirement.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization


{'runId': 'AutoML_27a228e2-7485-4cd2-b961-70749e5dba68',
 'target': 'amlcompute1',
 'status': 'Completed',
 'startTimeUtc': '2022-08-20T14:08:55.452254Z',
 'endTimeUtc': '2022-08-20T14:18:50.60396Z',
 'services': {},
 'properties': {'num_iterations': '10',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0.3',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'amlcompute1',
  'AMLSettingsJsonString': '{"path":null,"name":"credit_automl_experiment","subscription_id":"7b1b43ca-4b64-43cf-9446-edb35a04d7d1","resource_group":"defaultsrg01","workspace_name":"defaultscredits","region":"eastus","compute_target":"amlcompute1","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":true,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":true,"iterations":10,"primary_metric":"accuracy","ta

In [13]:
run.get_best_child(metrics='F1Score')

Experiment,Id,Type,Status,Details Page,Docs Page
credit_automl_experiment,AutoML_27a228e2-7485-4cd2-b961-70749e5dba68_8,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [14]:
#getting all metrics for the runs
for r in run.get_children():
    print("")
    print("Run ID :", r.id)
    print(run.get_metrics('accuracy'))
    print(run.get_metrics('F1Score'))
    print(run.get_metrics('AUC'))


Run ID : AutoML_27a228e2-7485-4cd2-b961-70749e5dba68_9
{'accuracy': 0.8211111111111111}
{}
{}

Run ID : AutoML_27a228e2-7485-4cd2-b961-70749e5dba68_8
{'accuracy': 0.8211111111111111}
{}
{}

Run ID : AutoML_27a228e2-7485-4cd2-b961-70749e5dba68_7
{'accuracy': 0.8211111111111111}
{}
{}

Run ID : AutoML_27a228e2-7485-4cd2-b961-70749e5dba68_6
{'accuracy': 0.8211111111111111}
{}
{}

Run ID : AutoML_27a228e2-7485-4cd2-b961-70749e5dba68_5
{'accuracy': 0.8211111111111111}
{}
{}

Run ID : AutoML_27a228e2-7485-4cd2-b961-70749e5dba68_4
{'accuracy': 0.8211111111111111}
{}
{}

Run ID : AutoML_27a228e2-7485-4cd2-b961-70749e5dba68_worker_1
{'accuracy': 0.8211111111111111}
{}
{}

Run ID : AutoML_27a228e2-7485-4cd2-b961-70749e5dba68_3
{'accuracy': 0.8211111111111111}
{}
{}

Run ID : AutoML_27a228e2-7485-4cd2-b961-70749e5dba68_2
{'accuracy': 0.8211111111111111}
{}
{}

Run ID : AutoML_27a228e2-7485-4cd2-b961-70749e5dba68_1
{'accuracy': 0.8211111111111111}
{}
{}

Run ID : AutoML_27a228e2-7485-4cd2-b961-70

In [16]:
#creating hyper drive paramers
from azureml.train.hyperdrive  import GridParameterSampling, choice

In [19]:
hyper_params=GridParameterSampling(
{'--n_estimators': choice(10,20,50,100),
'--min_samples_leaf': choice(1,2,5),
})

In [22]:
#configure hyperdrive class
from azureml.train.hyperdrive import HyperDriveConfig, PrimaryMetricGoal
hyper_config=HyperDriveConfig(run_config=script_config,
hyperparameter_sampling=hyper_params,
policy=None, 
primary_metric_name='accuracy',
primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
max_total_runs=20,max_concurrent_runs=2)




NameError: name 'script_config' is not defined

In [1]:
s='hello'
s[2:4]

'll'

In [ ]:
ws.get.datasets.get('credit_dataset')